# LU разложение, блочные и ленточные матрицы

Существует множество типов матриц, удовлетворяющих некоторым дополнительным условиям, для которых многие матричные операции могут быть вычислены быстрее или точнее, чем для матриц произвольного вида. 
В данной лабораторной мы начнем писать библиотеку на Python, которая будет содержать классы, реализующие базовые алгоритмы для работы с основными типами матриц.
Далее приводится исходный код класса `Matrix`, являющегося общим предком для всех матриц, и реализующего логику работы с матрицами общего вида.
Нижеследующий класс `FullMatrix` реализует хранилище для заполненных матриц. 
Изучите эти реализации и выполните следующие задания:


4. Реализуйте LUP разложение с перестановкой строк. Предъявите матрицу, на которой LUP разложение работает, а LU - нет.
5. Реализуйте метод прогонки и реализуйте метод `Matrix.solve` для решения линейных систем уравнений.
6. Реализуйте класс `SymmetricMatrix`, хранящий симметричные матрицы. Убедитесь, что метод `Matrix.lu` корректно работает с этим классом. Модифицируйте этот метод для класса `SymmetricMatrix` так, чтобы он использовал симметричность матрицы и работал в два раза быстрее.
7. Как влияет симметричность матрицы на устойчивость LU разложения?
8. Реализуйте класс `BandMatrix` для хранения ленточных матриц. Убедитесь в работоспособности методов `lu` и `solve`.
9. Воспользуйтесь реализованными классами для решения уравнения Пуассона $\Delta f=g$, использую операцию Лапласа из предыдущей лабораторной.

### Done

1. Напишите метод `lu` для класса `Matrix`, выполняющий LU разложение. 
2. Реализуйте метод `det`, вычисляющий определитель матрицы, опираясь на LU разложение.
3. Реализация `FullMatrix` может содержать своими элементами другие матрицы, т.е. описывать блочную матрицу. Убедитесь, что ваша реализация LU разложения работает с блочными матрицами.3. Реализация `FullMatrix` может содержать своими элементами другие матрицы, т.е. описывать блочную матрицу. Убедитесь, что ваша реализация LU разложения работает с блочными матрицами.

In [37]:
import numpy as np
from fractions import Fraction

In [52]:
class TextBlock:
    def __init__(self, rows):
        assert isinstance(rows, list)
        self.rows = rows
        self.height = len(self.rows)
        self.width = max(map(len,self.rows))
        
    @classmethod
    def from_str(_cls, data):
        assert isinstance(data, str)
        return TextBlock( data.split('\n') )
        
    def format(self, width=None, height=None):
        if width is None: width = self.width
        if height is None: height = self.height
        return [f"{row:{width}}" for row in self.rows]+[' '*width]*(height-self.height)
    
    @staticmethod
    def merge(blocks):
        return [" ".join(row) for row in zip(*blocks)]

class Matrix:
    """Общий предок для всех матриц."""
    @property
    def shape(self):
        raise NotImplementedError
    
    @property
    def dtype(self):
        raise NotImplementedError
    
    @property 
    def width(self):
        return self.shape[1]
    
    @property 
    def height(self):
        return self.shape[0]    
        
    def __repr__(self):
        """Возвращает текстовое представление для матрицы."""
        text = [[TextBlock.from_str(f"{self[r,c]}") for c in range(self.width)] for r in range(self.height)]
        width_el = np.array(list(map(lambda row: list(map(lambda el: el.width, row)), text)))
        height_el = np.array(list(map(lambda row: list(map(lambda el: el.height, row)), text)))
        width_column = np.max(width_el, axis=0)
        width_total = np.sum(width_column)
        height_row = np.max(height_el, axis=1)
        result = []
        for r in range(self.height):
            lines = TextBlock.merge(text[r][c].format(width=width_column[c], height=height_row[r]) for c in range(self.width))
            for l in lines:
                result.append(f"| {l} |")
            if len(lines)>0 and len(lines[0])>0 and lines[0][0]=='|' and r<self.height-1:
                result.append(f'| {" "*(width_total+self.width)}|')
        return "\n".join(result)
    
    def empty_like(self, width=None, height=None):
        raise NotImplementedError
        
    def ident_like(self):
        raise NotImplementedError
    
    def __getitem__(self, key):
        raise NotImplementedError
    
    def __setitem__(self, key, value):
        raise NotImplementedError
        
    def __add__(self, other):
        if isinstance(other, Matrix):
            assert self.width==other.width and self.height==other.height, f"Shapes does not match: {self.shape} != {other.shape}"
            matrix = self.empty_like()
            for r in range(self.height):
                for c in range(self.width):
                    matrix[r,c] = self[r,c] + other[r,c]
            return matrix
        return NotImplemented
    
    def __sub__(self, other):
        if isinstance(other, Matrix):
            assert self.width==other.width and self.height==other.height, f"Shapes does not match: {self.shape} != {other.shape}"
            matrix = self.empty_like()
            for r in range(self.height):
                for c in range(self.width):
                    matrix[r,c] = self[r,c] - other[r,c]
            return matrix
        return NotImplemented

    def __mul__(self, other):
        return self.__matmul__(other)
    
    def __matmul__(self, other):
        if isinstance(other, Matrix):
            assert self.width==other.height, f"Shapes does not match: {self.shape} != {other.shape}"
            matrix = self.empty_like()
            for r in range(self.height):
                for c in range(other.width):
                    acc = None
                    for k in range(self.width):
                        add = self[r,k]*other[k,c]
                        acc = add if acc is None else acc+add
                    matrix[r,c] = acc
            return matrix
        return NotImplemented
    
    def __truediv__(self, other):
        if isinstance(other, Matrix):
            assert self.width==other.width and self.height==other.height, f"Shapes does not match: {self.shape} != {other.shape}"
            divider = self.inverse()
            matrix = self*divider
            return matrix
        return NotImplemented
    
    def inverse(self):
        raise NotImplementedError

    def invert_element(self, element):
        if isinstance(element, (int, float, np.int32, np.float32)):
            return 1/element
        if isinstance(element, Fraction):
            return 1/element
        if isinstance(element, Matrix):
            return element.inverse()
        raise TypeError

    def lu(self):
        assert self.width==self.height, f"Matrix is not square: {self.height} != {self.width}"
        u = self.zero(self.width,self.height,self[0,0]-self[0,0])
        l = self.zero(self.width,self.height,self[0,0]-self[0,0])
        for i in range(self.height):
            if self[i,i]!=0:
                l[i,i] = self[0,0]/self[0,0]
        for i in range(self.height):
            for j in range(self.height):
                if i<=j:
                    temp = u.zero(u.width,u.height,u[0,0]-u[0,0])
                    for k in range(i+1):
                        temp[i,j] = temp[i,j]+l[i,k]*u[k,j]
                    u[i,j] = self[i,j]-temp[i,j]
                if i>j:
                    temp = u.zero(u.width,u.height,u[0,0]-u[0,0])
                    for k in range(j+1):
                        temp[i,j] = temp[i,j] + l[i,k]*u[k,j]
                    l[i,j] = (self[i,j]-temp[i,j])*u.invert_element(u[j,j])
        return l,u
    
    def det(self):
        assert self.width==self.height, f"Matrix is not square: {self.height} != {self.width}"
        l,u = self.lu()
        det = 1
        for i in range(u.height):
            det *= u[i,i]
        return det
    
    def lup(self):
        temp = self
        p = self.zero(self.width,self.height,self[0,0]-self[0,0])
        for i in range(self.height):
            if self[i,i]!=0:
                p[i,i] = self[0,0]/self[0,0]
        for i in range(self.height):
            ref_val = 0
            ref_num = -1
            for j in range(i,self.width):
                if np.abs(temp[j,i])>ref_val:
                    ref_val = np.abs(temp[j,i])
                    ref_num = j
            if ref_val!=0:
                temp.swap_rows(ref_num,i)
                p.swap_rows(ref_num,i)
                for j in range(i+1,self.height):
                    temp[j,i]/=temp[i,i]
                    for k in range(i+1,self.height):
                        temp[j,k] -= temp[j,i]*temp[i,k]
        u = self.zero(self.width,self.height,self[0,0]-self[0,0])
        l = self.zero(self.width,self.height,self[0,0]-self[0,0])
        for i in range(self.height):
            for j in range(self.height):
                if i==j:
                    u[i,j] = temp[i,j]
                    l[i,j] = 1
                elif i<j:
                    u[i,j] = temp[i,j]
                    l[i,j] = 0
                else:
                    l[i,j] = temp[i,j]
                    u[i,j] = 0
        return temp,l,u,p
        
    def swap_rows(self, num1, num2):
        matrix = self
        temp = self.empty_like()
        for i in range(self.height):
            temp[num1, i] = self[num1, i]
            matrix[num1, i] = self[num2, i]
            matrix[num2, i] = temp[num1, i]
        return matrix 
    
    def swap_cols(self, num1, num2):
        matrix = self
        temp = self.empty_like()
        for i in range(self.width):
            temp[i, num1] = self[i, num1]
            matrix[i, num1] = self[i, num2]
            matrix[i, num2] = temp[i, num1]
        return matrix 
    
    def transpone(self):
        raise NotImplementedError

    def zero(self, width, height, param):
        raise NotImplementedError


class FullMatrix(Matrix):
    """
    Заполненная матрица с элементами произвольного типа.
    """
    def __init__(self, data):
        """
        Создает объект, хранящий матрицу в виде np.ndarray `data`.
        """
        assert isinstance(data, np.ndarray)
        self.data = data
        
    def transpone(self):
        matrix = self.zero(self.width,self.height,self[0,0]-self[0,0])
        for i in range(self.width):
            for j in range(self.height):
                matrix[i,j]  = self[j,i]
        return matrix

    def empty_like(self, width=None, height=None):
        dtype = self.data.dtype
        if width is None:
            width = self.data.shape[1]
        if height is None:
            height = self.data.shape[0]       
        data = np.empty((height,width), dtype=dtype)
        return FullMatrix(data)
    
    def inverse(self):
        l,u = self.lu()
        l_inv = l.zero(l.width,l.height,l[0,0]-l[0,0])
        u_inv = u.zero(u.width,u.height,u[0,0]-u[0,0])
        for i in range(self.height):
            for j in range(self.width):
                if i>j:
                    u_inv[i,j] = 0
                    temp = u.zero(u.width,u.height,u[0,0]-u[0,0])
                    for k in range(i-1):
                        temp[i,j] = l[i,k]*l[k,j]
                    l_inv[i,j] = -l.invert_element(l[i,j])*temp[i,j]
                if i<j:
                    l_inv[i,j] = 0
                    temp = l.zero(l.width,l.height,l[0,0]-l[0,0])
                    for k in range(i-1):
                        temp[i,j] = u[i,k]*u[k,j]
                    u_inv[i,j] = -u.invert_element(u[i,j])*temp[i,j]
                else:
                    l_inv[i,j] = l.invert_element(l[i,i])
                    u_inv[i,j] = u.invert_element(u[i,i])
        return u_inv*l_inv
                    
    
    def ident_like(self, width=None, height=None):
        dtype = self.data.dtype
        if width is None:
            width = self.data.shape[1]
        if height is None:
            height = self.data.shape[0]       
        data = np.empty((height,width), dtype=dtype)
        for i in range(self.height):
            data[i,i] = self[i,i]*self.invert_element(self[i,i])
        return FullMatrix(data)
        
    @classmethod
    def zero(_cls, height, width, default=0):
        """
        Создает матрицу размера `width` x `height` со значениями по умолчанию `default`.
        """
        data = np.empty((height, width), dtype=type(default))
        data[:] = default
        return FullMatrix(data)
                    
    @property
    def shape(self):
        return self.data.shape
    
    @property
    def dtype(self):
        return self.data.dtype
        
    def __getitem__(self, key):
        row, column = key
        return self.data[row, column]
    
    def __setitem__(self, key, value):
        row, column = key
        self.data[row, column] = value
        

In [53]:
m = FullMatrix.zero(3,5,0)
print(m)
print(m.transpone())
print(m.shape)
print(m.dtype)
print(m.swap_rows(1,2))

| 0 0 0 0 0 |
| 0 0 0 0 0 |
| 0 0 0 0 0 |
| 0 0 0 |
| 0 0 0 |
| 0 0 0 |
| 0 0 0 |
| 0 0 0 |
(3, 5)
int32
| 0 0 0 0 0 |
| 0 0 0 0 0 |
| 0 0 0 0 0 |


#### Невырожденные матрицы

In [54]:
m = FullMatrix.zero(3,3,Fraction(1,2))
for i in range(m.height):
    for j in range(m.width):
        m[i,j] = Fraction(i+1,j+1)+i*i+j*j
d = FullMatrix.zero(3,3,Fraction(0,1))
for i in range(min(d.height,d.width)):
        d[i,i] = Fraction(i,1)        
'''print(m)
print(m.shape)
print(m.dtype)
print(f"m+m", m+m)
print(f"m*d", m*d)
'''
print(m)
print("l=")
print(m.lu()[0])
print("u=")
print(m.lu()[1])
print("a=LU=")
print(m.lu()[0]*m.lu()[1])
print(m.det())

| 1 3/2  13/3 |
| 3 3    17/3 |
| 7 13/2 9    |
l=
| 1 0   0 |
| 3 1   0 |
| 7 8/3 1 |
u=
| 1 3/2  13/3  |
| 0 -3/2 -22/3 |
| 0 0    -16/9 |
a=LU=
| 1 3/2  13/3 |
| 3 3    17/3 |
| 7 13/2 9    |
8/3


In [55]:
matr = FullMatrix.zero(2,2,m)
print(matr)
print(matr.lu()[0])
print(matr.lu()[1])

| | 1 3/2  13/3 | | 1 3/2  13/3 | |
| | 3 3    17/3 | | 3 3    17/3 | |
| | 7 13/2 9    | | 7 13/2 9    | |
|                                 |
| | 1 3/2  13/3 | | 1 3/2  13/3 | |
| | 3 3    17/3 | | 3 3    17/3 | |
| | 7 13/2 9    | | 7 13/2 9    | |
| | -133/16 -47/8   -39/16 | | 0 0 0 |                  |
| | -169/16 -67/8   -51/16 | | 0 0 0 |                  |
| | -809/48 -347/24 -81/16 | | 0 0 0 |                  |
|                                                       |
| | -133/16 -47/8   -39/16 | | -133/16 -47/8   -39/16 | |
| | -169/16 -67/8   -51/16 | | -169/16 -67/8   -51/16 | |
| | -809/48 -347/24 -81/16 | | -809/48 -347/24 -81/16 | |
| | 1 3/2  13/3 | | 1 3/2  13/3 |           |
| | 3 3    17/3 | | 3 3    17/3 |           |
| | 7 13/2 9    | | 7 13/2 9    |           |
|                                           |
| | 0 0 0 |       | 44    759/16  1147/12 | |
| | 0 0 0 |       | 61    1035/16 1531/12 | |
| | 0 0 0 |       | 308/3 1729/16 7543/36 | |


#### Вырожденные матрицы

In [56]:
m = FullMatrix.zero(5,5,Fraction(1,2))
for i in range(m.height):
    for j in range(m.width):
        m[i,j] = Fraction(i+1,j+1)+i
d = FullMatrix.zero(3,3,Fraction(0,1))
for i in range(min(d.height,d.width)):
        d[i,i] = Fraction(i,1)        
'''print(m)
print(m.shape)
print(m.dtype)
print(f"m+m", m+m)
print(f"m*d", m*d)
'''
print(m)
print(m.det())

| 1 1/2  1/3  1/4  1/5  |
| 3 2    5/3  3/2  7/5  |
| 5 7/2  3    11/4 13/5 |
| 7 5    13/3 4    19/5 |
| 9 13/2 17/3 21/4 5    |


ZeroDivisionError: Fraction(1, 0)

### LUP

In [61]:
m = FullMatrix.zero(3,3,Fraction(1,2))
for i in range(m.height):
    for j in range(m.width):
        m[i,j] = Fraction(i+1,j+1)+i*i+j*j
print(m)
print("det",m.det())
temp,l,u,p = m.lup()
print("temp")
print(temp)
print("l")
print(l)
print("u")
print(u)
print("p")
print(p)
m_new = p.transpone()*l*u
print(m_new)

| 1 3/2  13/3 |
| 3 3    17/3 |
| 7 13/2 9    |
det 8/3
temp
| 7   13/2 9     |
| 1/7 4/7  64/21 |
| 3/7 3/8  2/3   |
l
| 1   0   0 |
| 1/7 1   0 |
| 3/7 3/8 1 |
u
| 7 13/2 9     |
| 0 4/7  64/21 |
| 0 0    2/3   |
p
| 0 0 1 |
| 1 0 0 |
| 0 1 0 |
| 1 3/2  13/3 |
| 3 3    17/3 |
| 7 13/2 9    |
